In [145]:
# Dependencies and Setup below
import pandas as pd
import numpy as np

# File to Load 
purchase_data = "PymoliData.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv("PymoliData.csv")

#print the data
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [146]:
#Count the number of unique players in SN. Need to get unique values
total_player_count = pd.DataFrame({"Total Players":[purchase_data["SN"].nunique()]})
total_player_count

,Total Players
0,576


In [147]:
#Purchasing Analysis (Total)
#Run basic calculations to obtain number of unique items, average price, etc.
#Create a summary data frame to hold the results

num_unique_items = purchase_data["Item Name"].nunique()
average_price=purchase_data["Price"].mean()
average_price="${:,.2f}".format(average_price)
number_of_purchases=purchase_data["Purchase ID"].count()
total_revenue=(purchase_data["Price"]).sum()
total_revenue="${:,.2f}".format(total_revenue)

#Create a summary data frame to hold the results
summary_table=pd.DataFrame({"Number of Unique Items":[num_unique_items], 
                            "Average Price":[average_price],
                            "Number of Purchases": [number_of_purchases],
                            "Total Revenue": [total_revenue]})

summary_table=summary_table.reset_index(drop=True)                          
#Optional: give the displayed data cleaner formatting
#Display the summary data frame
summary_table.head()

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [148]:
#Gender Demographics
#Group data by gender and obtain counts for all three Gender categories
purchase_by_gender=purchase_data.groupby(["Gender"])
total_players_gender=purchase_by_gender["SN"].nunique()


#Find the percentage of players in each category
total_players_percent=total_players_gender / purchase_data["SN"].nunique() * 100


#merge dataframes into full analysis, give series column names here
total_player_count_gender = pd.concat([total_players_gender.rename("Total Player Count"),
                                       total_players_percent.rename("Total Percent")],axis=1)

#format for nice display
#remove index for cleaner display
total_player_count_gender.index.name = None

#sort, format percent column, output result
total_player_count_gender.sort_values(["Total Player Count"], ascending = False).style.format({"Total Percent":'{:.2f}%'})

,Total Player Count,Total Percent
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [149]:
#Purchasing Analysis (Gender)
#Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender
purchase_count_gender=purchase_by_gender["Item ID"].count()
average_purchase_price_by_gender=round(purchase_by_gender["Price"].mean(),2)
total_purchase_per_person=purchase_by_gender["Price"].sum()
average_purchase_per_person=total_purchase_per_person/total_players_gender

#Create a summary data frame to hold the results
gender_purchase_summary=pd.concat([purchase_count_gender.rename("Purchase Count"),
                                   average_purchase_price_by_gender.rename("Average Purchase Price"),
                                   total_purchase_per_person.rename("Total Purchase Value"),average_purchase_per_person.rename("Avg Total Purchase per Person")],
                                  axis=1)
#Format to make it look nice
gender_purchase_summary['Average Purchase Price'] = gender_purchase_summary['Average Purchase Price'].astype(float).map("\${:,.2f}".format)
gender_purchase_summary['Total Purchase Value'] = gender_purchase_summary['Total Purchase Value'].astype(float).map("\${:,.2f}".format)
gender_purchase_summary['Avg Total Purchase per Person'] = gender_purchase_summary['Avg Total Purchase per Person'].astype(float).map("\${:,.2f}".format)

#Display the summary data frame
gender_purchase_summary

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,\$3.20,\$361.94,\$4.47
Male,652,\$3.02,"\$1,967.64",\$4.07
Other / Non-Disclosed,15,\$3.35,\$50.19,\$4.56


In [150]:
#Age Demographics

#Establish bins for ages
bins = [0, 9, 14, 19, 24, 29, 34, 39, 200]
labels = ['<10', '10-14', '15-19', '20-24', '26-29', '30-34', '35-39', '40+']

#Categorize the existing players using the age bins and add to the original dataframe
purchase_data["Age Group"] = pd.cut(purchase_data['Age'], bins, labels = labels)
purchase_data_by_age =  purchase_data.groupby("Age Group")

#Calculate the numbers and percentages by age group
total_count_by_age=purchase_data_by_age["SN"].nunique()
percent_by_age=total_count_by_age / purchase_data["SN"].nunique()

#Create a summary data frame to hold the results
age_summary = pd.concat([total_count_by_age.rename("Total Count"),
                        percent_by_age.rename("Percent of Players")],axis=1)

#Optional: round the percentage column to two decimal points
age_summary['Percent of Players'] = age_summary['Percent of Players'].astype(float).map("{:.2%}".format)

#Display Age Demographics Table
age_summary

,Total Count,Percent of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
26-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [151]:
#Purchasing Analysis (Age)

#Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below
purchase_count_by_age=purchase_data_by_age["Item ID"].count()
average_purchase_price_by_age=round(purchase_data_by_age["Price"].mean(),2)
total_purchase_per_age_group=purchase_data_by_age["Price"].sum()
average_purchase_per_age_group=total_purchase_per_age_group/purchase_count_by_age

#Create a summary data frame to hold the results
age_analysis = pd.concat([purchase_count_by_age.rename("Purchase Count"),
                         average_purchase_price_by_age.rename("Average Purchase Price"),
                         total_purchase_per_age_group.rename("Total Purchase Value"),
                         average_purchase_per_age_group.rename("Avg Total Purchase per Person")],
                         axis=1)

#Optional: give the displayed data cleaner formatting
age_analysis['Average Purchase Price'] = age_analysis['Average Purchase Price'].astype(float).map("\${:,.2f}".format)
age_analysis['Total Purchase Value'] = age_analysis['Total Purchase Value'].astype(float).map("\${:,.2f}".format)
age_analysis['Avg Total Purchase per Person'] = age_analysis['Avg Total Purchase per Person'].astype(float).map("\${:,.2f}".format)

#Display the summary data frame
age_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,23,\$3.35,\$77.13,\$3.35
10-14,28,\$2.96,\$82.78,\$2.96
15-19,136,\$3.04,\$412.89,\$3.04
20-24,365,\$3.05,"\$1,114.06",\$3.05
26-29,101,\$2.90,\$293.00,\$2.90
30-34,73,\$2.93,\$214.00,\$2.93
35-39,41,\$3.60,\$147.67,\$3.60
40+,13,\$2.94,\$38.24,\$2.94


In [152]:
#Top Spenders
#Run basic calculations to obtain the results
top_spenders_names=purchase_data.groupby("SN")
top_count=top_spenders_names["Item ID"].count()
total_purchase_by_sn=top_spenders_names["Price"].sum()
avg_purchase_by_sn=total_purchase_by_sn/top_count

#Create a summary data frame to hold the results
sn_analysis = pd.concat([top_count.rename("Purchase Count"),
                         avg_purchase_by_sn.rename("Average Purchase Price"),
                         total_purchase_by_sn.rename("Total Purchase Value")],
                         axis=1)

#Optional: give the displayed data cleaner formatting
sn_analysis['Average Purchase Price'] = sn_analysis['Average Purchase Price'].astype(float).map("\${:,.2f}".format)
sn_analysis['Total Purchase Value'] = sn_analysis['Total Purchase Value'].astype(float).map("\${:,.2f}".format)

#Sort the total purchase value column in descending order
sn_analysis.sort_values(["Total Purchase Value"], ascending = False)


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Haillyrgue51,3,\$3.17,\$9.50
Phistym51,2,\$4.75,\$9.50
Lamil79,2,\$4.64,\$9.29
Aina42,3,\$3.07,\$9.22
Saesrideu94,2,\$4.59,\$9.18
Arin32,2,\$4.54,\$9.09
Rarallo90,3,\$3.02,\$9.05
Baelollodeu94,2,\$4.51,\$9.03
Aelin32,3,\$2.99,\$8.98


In [153]:
#Most Popular Items
#Retrieve the Item ID, Item Name, and Item Price columns
item_df=purchase_data[['Item ID','Item Name', 'Price']]

#Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value
item_df=item_df.groupby(["Item ID", "Item Name"])
item_count=item_df["Item ID"].count()
item_total=item_df["Price"].sum()
item_cost=item_total / item_count

#Create a summary data frame to hold the results
item_summary = pd.concat([item_count.rename("Purchase Count"),
                          item_cost.rename("Item Price"), 
                          item_total.rename("Total Purchase Value")], 
                         axis=1)

#Optional: give the displayed data cleaner formatting
item_summary['Item Price'] = item_summary['Item Price'].astype(float).map("\${:,.2f}".format)
item_summary['Total Purchase Value'] = item_summary['Total Purchase Value'].astype(float).map("\${:,.2f}".format)

#Display a preview of the summary data frame
#Sort the purchase count column in descending order
item_summary.sort_values(["Purchase Count"], ascending = False)


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,\$4.23,\$50.76
145,Fiery Glass Crusader,9,\$4.58,\$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,\$3.53,\$31.77
82,Nirvana,9,\$4.90,\$44.10
19,"Pursuit, Cudgel of Necromancy",8,\$1.02,\$8.16
103,Singed Scalpel,8,\$4.35,\$34.80
75,Brutality Ivory Warmace,8,\$2.42,\$19.36
72,Winter's Bite,8,\$3.77,\$30.16
60,Wolf,8,\$3.54,\$28.32


In [156]:
#Most Profitable Items
item_summary2 = pd.concat([item_count.rename("Purchase Count"),
                          item_cost.rename("Item Price"), 
                          item_total.rename("Total Purchase Value")], 
                         axis=1)
#Sort the above table by total purchase value in descending order
item_summary2.sort_values(["Total Purchase Value"], ascending=False).head().style.format({'Item Price': "\${:,.2f}", 'Total Purchase Value': "\${:,.2f}"})

#Optional: give the displayed data cleaner formatting
#Display a preview of the data frame


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,\$4.23,\$50.76
82,Nirvana,9,\$4.90,\$44.10
145,Fiery Glass Crusader,9,\$4.58,\$41.22
92,Final Critic,8,\$4.88,\$39.04
103,Singed Scalpel,8,\$4.35,\$34.80
